# FAISS를 활용법과 ODQA 완성하기
이전 과제 마지막 질문은 팀원들과 고민해보셨나요? 아무래도 동시에 많은 사용자가 Query 를 보내는 상황을 고려했을 때, GPU 를 사용하더라도 유사한 Passage 를 빠르게 찾아서 반환하기에는 시간이 조금 걸릴 것 같네요. 그 해답을 이번 6강에서 배운 Faiss 를 통해 해결해봅시다.

그러면 우리는 성공적으로 빠르고 정확하게 Passage Retrieval 을 할 수 있게 되었습니다. 하지만 QA 모델을 완성하려면, 찾아낸 Passage 에서 답을 찾아내는 과정도 필요하겠죠. Retrieval 된 Passage 로부터 답을 찾아내는 Reader 모델까지 연결시키는 코드를 마무리해봅시다.

```
🛠 Setup을 하는 부분입니다. 이전 과제에서 반복되는 부분이기 때문에 무지성 실행 하셔도 좋습니다.
💻 실습 코드입니다. 따라가면서 코드를 이해해보세요.
💫 과제 정답입니다. 정답을 보고 내가 작성한 코드와 비교해보세요.
```

## 🛠 초기설정

### 🛠 Requirements

In [ ]:
!pip install tqdm==4.48.0 -q
!pip install datasets==1.4.1 -q
!pip install transformers==4.5.0 -q
!pip install faiss-cpu -q

### 🛠 난수 고정 및 버전 확인

In [ ]:
import random
import numpy as np
import pandas as pd
from tqdm.auto import tqdm, trange
from pprint import pprint

import faiss

import torch
from torch.utils.data import RandomSampler, DataLoader, TensorDataset
import torch.nn.functional as F

from datasets import load_dataset
from transformers import (
    AutoTokenizer,
    BertModel, BertPreTrainedModel,
    AdamW, get_linear_schedule_with_warmup,
    TrainingArguments,
)

In [ ]:
# 난수 고정
def set_seed(random_seed):
    torch.manual_seed(random_seed)
    torch.cuda.manual_seed(random_seed)
    torch.cuda.manual_seed_all(random_seed)  # if use multi-GPU
    random.seed(random_seed)
    np.random.seed(random_seed)
    
set_seed(42) # magic number :)

In [ ]:
print ("PyTorch version:[%s]."%(torch.__version__))
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print ("device:[%s]."%(device))

PyTorch version:[1.9.0].
device:[cuda:0].


### 🛠 데이터셋 로딩
KorQuAD train 데이터셋을 학습 데이터로 활용

In [ ]:
from datasets import load_dataset

dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
print(f'총 {len(corpus)}개의 지문이 있습니다.')

Reusing dataset squad_kor_v1 (C:\Users\pha\.cache\huggingface\datasets\squad_kor_v1\squad_kor_v1\1.0.0\31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7)


총 9606개의 지문이 있습니다.


저번 실습에서는 DPR 구현에 초점을 뒀기 때문에 소량의 데이터만 활용했습니다. 이번에는 실습에서는 Faiss를 통해 대량의 Passage들과 유사도를 구해야하므로, 전체 Validation 데이터를 활용합니다.

In [ ]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
print(f'총 {len(search_corpus)}개의 지문이 있습니다.')

총 960개의 지문이 있습니다.


### 🛠 토크나이저 준비 - Huggingface 제공 tokenizer 이용

BERT를 encoder로 사용하므로, KLUE에서 제공하는 `klue/bert-base` tokenizer를 활용해봅시다. 다른 pretrained 모델을 사용하고 싶으시다면, `model_checkpoint`를 바꿔보세요 !

In [ ]:
from transformers import AutoTokenizer

model_checkpoint = "klue/bert-base"

tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

### 🛠 Dense Passage Retrieval 코드 가져오기
Faiss를 사용하려면 우선 Passage들이 전부 임베딩 되어있어야겠죠? 이 과정은 저번 실습 코드에서 Dense Retriever를 활용해봅시다. 여러분이 완성하신 코드가 있으면 직접 활용해보세요. 없다면 저희가 제공드린 코드를 활용하셔도 무관합니다.

In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
    def __init__(self, config):
        super(BertEncoder, self).__init__(config)

        self.bert = BertModel(config)
        self.init_weights()
      
    def forward(
            self,
            input_ids, 
            attention_mask=None,
            token_type_ids=None
        ): 
  
        outputs = self.bert(
            input_ids,
            attention_mask=attention_mask,
            token_type_ids=token_type_ids
        )
        
        pooled_output = outputs[1]
        return pooled_output

# Pre-train된 모델을 사용해줍니다. 위에서 사용한 `model_checkpoint`를 재활용합니다.
p_encoder = BertEncoder.from_pretrained(model_checkpoint)
q_encoder = BertEncoder.from_pretrained(model_checkpoint)

if torch.cuda.is_available():
    p_encoder.cuda()
    q_encoder.cuda()

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at klue/bert-base were not used when initializing BertEncoder: ['cls.predictions.bi

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(
    search_corpus,
    padding="max_length",
    truncation=True,
    return_tensors="pt"
)
valid_dataset = TensorDataset(
    valid_p_seqs["input_ids"],
    valid_p_seqs["attention_mask"],
    valid_p_seqs["token_type_ids"]
)
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(
    valid_dataset,
    sampler=valid_sampler,
    batch_size=eval_batch_size
)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

    epoch_iterator = tqdm(
        valid_dataloader,
        desc="Iteration",
        position=0,
        leave=True
    )
    p_encoder.eval()

    for _, batch in enumerate(epoch_iterator):
        batch = tuple(t.cuda() for t in batch)

        p_inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2]
        }
        
        outputs = p_encoder(**p_inputs).to("cpu").numpy()
        p_embs.extend(outputs)

Iteration: 100%|██████████| 120/120 [00:26<00:00,  4.46it/s]


In [ ]:
p_embs = np.array(p_embs)
p_embs.shape # (num_passage, emb_dim)

(960, 768)

Question encoder를 활용해여 question dense embedding 생성

In [ ]:
# np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['천안함 침몰 사건을 패전이라 지적한 것은 누구인가?',
 '고종은 연호를 무엇으로 고쳤는가?',
 '영락경에서 10신위의 수행자를 간단히 칭하는 말은?',
 '김 당선자의 조치가 국가 화합과 경제위기 극복 등에 도움이 될 것이라고 보도한 신문사는?',
 '2007년에  밴드는 더 폴리스 외에 어떤 밴드의 서포팅 밴드로써 공연을 하였는가?']

In [ ]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
    q_encoder.eval()
    q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()
q_embs.shape  # (num_query, emb_dim)

(5, 768)

## 💫 Faiss를 활용한 Retriever를 저번 과제처럼 Class로 짜주세요.
아래는 기능만 구현한 코드입니다.

In [ ]:
class FaissRetrieval:
    def __init__(self, p_embs, num_clusters=16):

        """
        Arguments:
            p_embs (torch.Tensor):
                위에서 사용한 Passage Encoder로 구한
                전체 Passage들의 Dense Representation을 받아옵니다.
                
        Summary:
            초기화하는 부분
            `build_faiss` 메소드도 여기서 수행하면 좋을 것 같습니다.
        """
        
        self.p_embs = p_embs
        self.build_faiss(num_clusters=num_clusters)

    def build_faiss(self, num_clusters=16):

        """
        Note:
            위에서 Faiss를 사용했던 기억을 떠올려보면,
            Indexer를 구성해서 .search() 메소드를 활용했습니다.
            여기서는 Indexer 구성을 해주도록 합시다.
        """

        emb_dim = self.p_embs.shape[-1]

        quantizer = faiss.IndexFlatL2(emb_dim)
        self.indexer = faiss.IndexIVFScalarQuantizer(
            quantizer,
            quantizer.d,
            num_clusters,
            faiss.METRIC_L2
        )
        self.indexer.train(self.p_embs)
        self.indexer.add(self.p_embs)

    def get_relevant_doc(self, q_emb, k=1):
        """
        Arguments:
            query (torch.Tensor):
                Dense Representation으로 표현된 query를 받습니다.
                문자열이 아님에 주의합시다.
            k (int, default=1):
                상위 몇 개의 유사한 passage를 뽑을 것인지 결정합니다.

        Note:
            받은 query를 이 객체에 저장된 indexer를 활용해서
            유사한 문서를 찾아봅시다.
        """
        
        q_emb = q_emb.astype(np.float32)
        D, I = self.indexer.search(q_emb, k)

        return D.tolist()[0], I.tolist()[0]

In [ ]:
query = "금강산의 겨울 이름은?"

valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors="pt").to("cuda")

with torch.no_grad():
    q_encoder.eval()
    q_embs = q_encoder(**valid_q_seqs).to("cpu").numpy()

# p_embs는 처음에 만든 embedding을 이용합시다.
retriever = FaissRetrieval(p_embs)
retriever.build_faiss()
results = retriever.get_relevant_doc(q_embs, k=5)

In [ ]:
print("[Search query]\n", query, "\n")

print(f"Top-{len(results[0])} passages")
for d, i in zip(*results):
    print(f"Distance {d:.5f} | Passage {i}")
    pprint(search_corpus[i])
print('\n')

[Search query]
 금강산의 겨울 이름은? 

Top-5 passages
Distance 325.80603 | Passage 826
('천사장(archangel)에 해당하는 영어 단어의 접두사(arch)는 “수석” 혹은 “우두머리”를 뜻하는 것으로 천사장 즉 수석 천사가 '
 '하나뿐임을 시사한다. 성경에서 “천사장”이 복수 형태로 나오는 경우는 결코 없다. 데살로니가 첫째 4:16에서는 천사장의 탁월함과 그 '
 '직무의 권위에 대해 말하면서 부활되신 주 예수 그리스도를 그런 식으로 부른다. “주께서 친히 호령과 천사장의 음성과 하느님의 나팔과 함께 '
 '하늘에서 내려오실 것이며, 그리스도와 결합하여 죽어 있는 사람들이 먼저 일어날 것[입니다].” 그러므로 “천사장”이란 단어와 직접 '
 '관련되어 있는 이름이 미가엘뿐이라는 사실은 의미 깊은 것이다.—유 9. 미가엘 1번 참조. (출처 : Insight on the '
 'Scriptures)')
Distance 344.57306 | Passage 790
('1979년에 “중화인민공화국 형법”이 제정되기까지는 단행 법령이나 각종 사법해석, 공산당의 문서 등에 형벌규칙을 두고 있었다. 1979년 '
 '형법은 범죄를 “사회에 위해를 가하는 행위로서, 법률에 의한 형벌을 받을 수 있는 것”이라고 정의하여, 유추해석을 공인하고 있다. '
 '1997년에 형법이 전면적으로 개정되었다. 그 이후, 전인대 상무위원회에 의한 다수의 개정이 있다. 1997년 형법은 유추해석을 '
 '금지하여, 죄형법정주의를 채택하였다. 한국, 일본을 비롯한 대륙법권의 형법과 비교할 때 큰 특색으로서는, 공범론에 있어서, 정범 · '
 '종범이라고 하는 구성요건을 중심으로 한 구조 대신에, 주범 · 종범이라고 하는 범죄의 경위에 착안한 구조를 이용하고 있다. 주형에는 '
 '관제(공안기관의 감독하에 생활하는 것), 구역(노동개조형), 유기징역, 무기징역, 사형의 5종류가 있고, 부가형으로는 벌금, 정치적 '
 '권리박탈, 재산 

## 💫 SparseRetrieval 완성하기
Baseline의 `retrieval.py` 코드를 참고해주세요. 수고 많으셨습니다 !

## ✔ 과제를 마무리하며 ...
과제의 난이도가 조금 있었는데 수행하시느라 고생 많으셨습니다. 대부분 과제에서 모듈화, 클래스화를 권장했는데요! 엔지니어로서 코드 디자인에 대한 감각을 익힐 뿐 아니라, 기계독해 대회의 베이스라인 이해도를 높이기 위해서였습니다. 실습과 과제를 성실히 수행한 후 베이스라인을 다시 보았을 때, 이해도가 올라가있는 여러분들을 발견하셨을 거라 믿습니다. 질문이 있으면 언제든 슬랙에 남겨주세요 ! 남은 대회기간, 부스트캠프 기간 모두 화이팅입니다!